In [4]:
# append path to find utils module in urbansim
import os 
import sys
cwd = os.getcwd() 
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(parentdir) # to get path to utils module



import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from database import get_connection_string

db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)

In [5]:
sched_dev_parcel_sql = '''
SELECT [site_id]
      ,[parcel_id]
      ,[capacity_3]
      ,[sfu_effective_adj]
      ,[mfu_effective_adj]
      ,[mhu_effective_adj]
      ,[notes]
      ,[editor]
  FROM [urbansim].[urbansim].[scheduled_development_parcel]
  WHERE capacity_3 > 0
'''

sched_dev_df = pd.read_sql(sched_dev_parcel_sql, mssql_engine)

In [7]:
yr_for_each_site = sched_dev_df.drop_duplicates(subset='site_id',keep='first').copy()
yr_for_each_site.head()
len(yr_for_each_site)

471

In [8]:
sites = sched_dev_df.site_id.unique()
len(sites)

471

In [9]:
#7, 4, 9
np.random.seed(9)
random_num = np.random.randint(1,21,size=(len(sites)))
# random_num

# OLD

# NEW

In [10]:
 d = {1: 2017, 2: 2018, 3: 2022, 4: 2023, 5: 2021, 6: 2022, 7: 2023, 8: 2024,\
      9: 2025, 10: 2025, 11: 2024, 12: 2021, 13: 2022, 14: 2023, 15: 2024,\
      16: 2019, 17: 2020, 18: 2021, 19: 2022, 20: 2018}
random_yr = [d.get(x) for x in random_num]
# random_yr

In [11]:
yr_for_each_site['yr'] = random_yr
yr_for_each_site = yr_for_each_site[['site_id','yr']].copy()
# yr_for_each_site.head()
pd.DataFrame({'sites_by_year': yr_for_each_site.
              groupby(["yr"]).site_id.size()}).reset_index()

,yr,sites_by_year
0,2017,16
1,2018,48
2,2019,22
3,2020,26
4,2021,82
5,2022,94
6,2023,66
7,2024,68
8,2025,49


In [12]:
sched_dev_w_year = pd.merge(sched_dev_df,yr_for_each_site, \
                            left_on='site_id',right_on='site_id',how='left')
# sched_dev_w_year.head()
pd.DataFrame({'units_by_year': sched_dev_w_year.
              groupby(["yr"]).capacity_3.sum()}).reset_index()

,yr,units_by_year
0,2017,6783.0
1,2018,7899.0
2,2019,6374.0
3,2020,6414.0
4,2021,8338.0
5,2022,12366.0
6,2023,9954.0
7,2024,10913.0
8,2025,9935.0


In [13]:
sched_dev_w_year['sched_version_id'] = 2

In [ ]:
sched_dev_for_db = sched_dev_w_year[['sched_version_id','parcel_id','yr',\
                                     'site_id','capacity_3']].copy()

In [ ]:
len(sched_dev_for_db)

In [ ]:
# sched_dev_for_db.head()

In [ ]:
sched_dev_for_db.capacity_3.sum()

In [ ]:
sched_dev_for_db.to_sql(name='scheduled_development_do_not_use', \
                        con=mssql_engine, schema='urbansim', index=False,if_exists='append')